In [1]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
import pandas as pd
import snowflake.connector as sf
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [2]:
# Define the S3 bucket and Snowflake parameters
snowflake_account = 'at67872.ap-south-1.aws'
snowflake_user = 'FYP4'
snowflake_password = '!Q2w3e4r5t6y'
warehouse='COMPUTE_WH'
snowflake_database = 'COAPDATA'
snowflake_schema = 'METER_SCHEMA'


In [77]:
model = load_model('model.h5')

In [6]:
# Connect to Snowflake
conn = sf.connect(
        user=snowflake_user,
        password=snowflake_password,
        account=snowflake_account,
        warehouse=warehouse,
        database=snowflake_database,
        schema=snowflake_schema
    )

In [7]:
# Get the list of water meter views
cursor = conn.cursor()
cursor.execute(f"SHOW VIEWS IN SCHEMA {snowflake_database}.{snowflake_schema}")
views = [row[1] for row in cursor.fetchall()]
cursor.close()

True

In [8]:
views[0]

'METER_1_VIEW'

In [9]:
# Dictionary to store the dataframes for each water meter view
view_dataframes = {}
dataframes = []

In [10]:
# Iterate over each water meter view
for view in views:
    # Retrieve the data from the current water meter view
    query = f"SELECT Date, total FROM {snowflake_database}.{snowflake_schema}.{view} ORDER BY Date DESC"
    df = pd.read_sql(query, conn)
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    df = pd.DataFrame(data, columns=columns)
    dataframes.append(df)
        
    # # # Scale the data using standardization
    # scaler = StandardScaler()
    # # Convert the 'total' column to a numpy array
    # data = df['TOTAL'].values 
    # scaled_data = scaler.fit_transform(data.reshape(-1, 1))
        
    # # # Store the dataframe in the dictionary
    # view_dataframes[view] = scaled_data

In [11]:
# Scale each dataframe using StandardScaler
scaler = StandardScaler()
scaled_dataframes = []

for df in dataframes:
    df = df['TOTAL'].values 
    scaled_data = scaler.fit_transform(df.reshape(-1, 1))
    # scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
    scaled_dataframes.append(scaled_data)

In [12]:
dataframes[0]

,DATE,TOTAL
0,2023-06-07,535
1,2023-06-01,250
2,2023-01-04,255
3,2022-01-04,245


In [13]:
# Access each scaled dataframe
df1_scaled = scaled_dataframes[0]
df1_scaled


array([[ 1.73134044],
       [-0.57711348],
       [-0.53661429],
       [-0.61761267]])

In [14]:
df2_scaled = scaled_dataframes[1]
df2_scaled

IndexError: list index out of range

In [15]:
import tensorflow as tf
from tensorflow.keras import layers


In [16]:
# Define the input layer
input_layer = Input(shape=(1,))

# Define the encoder layers
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)

# Define the decoder layers
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(1, activation='linear')(decoded)

# Create the autoencoder model
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [17]:
# List to store the trained autoencoders
autoencoders = []

# Define the autoencoder model
input_dim = scaled_dataframes[0].shape[1]
encoding_dim = 64

for scaled_df in scaled_dataframes:
    input_layer = layers.Input(shape=(input_dim,))
    encoder = layers.Dense(encoding_dim, activation='relu')(input_layer)
    decoder = layers.Dense(input_dim, activation='linear')(encoder)

    autoencoder = tf.keras.Model(input_layer, decoder)

    # Compile and train the autoencoder
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.fit(scaled_df, scaled_df, epochs=100, batch_size=32)

    # Add trained autoencoder to the list
    autoencoders.append(autoencoder)

Epoch 1/100
1/1 [==============================] - 1s 533ms/step - loss: 1.1708
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 1.1431
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 1.1160
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0891
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0627
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0367
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0110
Epoch 8/100
1/1 [==============================] - 0s 994us/step - loss: 0.9857
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.9609
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 0.9365
Epoch 11/100
1/1 [==============================] - 0s 3ms/step - loss: 0.9125
Epoch 12/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8888
Epoch 13/100
1/1 [==============================] - 0s 6m

In [18]:
anomaly_threshold = []  # Adjust this threshold as needed
threshold=[]

anomalies = []

# Iterate over each scaled dataframe and its corresponding autoencoder
for scaled_df, autoencoder in zip(scaled_dataframes, autoencoders):
    # Reconstruct the data using the autoencoder
    reconstructed_data = autoencoder.predict(scaled_df)

    # Calculate the reconstruction error
    reconstruction_error = np.mean(np.square(scaled_df - reconstructed_data), axis=1)

    # Calculate the threshold dynamically
    threshold = np.percentile(reconstruction_error, 95)
    print(threshold)

    # Identify anomalies based on the threshold
    view_anomalies = scaled_df[reconstruction_error > threshold]

    # Add the anomalies to the list
    anomalies.append(view_anomalies)


0.08174077590330205


In [19]:
# List to store anomaly flags
anomaly_flags = []

# Iterate over each view and its corresponding autoencoder and scaled dataframe
for autoencoder, scaled_df in zip(autoencoders, scaled_dataframes):
    # Get the last entry of the scaled dataframe
    last_entry = scaled_df[0]
    print (last_entry)

    # Reshape the last entry to match the autoencoder input shape
    last_entry = last_entry.reshape(1, -1)

    # Reconstruct the data using the autoencoder
    reconstructed_data = autoencoder.predict(last_entry)

    # Calculate the reconstruction error
    reconstruction_error = np.mean(np.square(last_entry - reconstructed_data))

    # Check if the reconstruction error exceeds the threshold for anomaly detection
    #anomaly_threshold = 0.01  # Adjust the threshold as needed
    is_anomaly = int(reconstruction_error > threshold)

    # Store the anomaly flag
    anomaly_flags.append(is_anomaly)

[1.73134044]


In [21]:
anomalies[1]

IndexError: list index out of range

In [22]:
anomaly_flags[0]

1

In [93]:
anomaly_flags[1]

0

In [95]:
# Define the S3 bucket and Snowflake parameters
snowflake_account = 'at67872.ap-south-1.aws'
snowflake_user = 'FYP4'
snowflake_password = '!Q2w3e4r5t6y'
warehouse='COMPUTE_WH'
snowflake_database = 'COAPDATA'
snowflake_schema = 'METER_SCHEMA'


In [96]:
# Establish connection to Snowflake
conn = sf.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    warehouse=warehouse,
    database=snowflake_database,
    schema=snowflake_schema
)

In [99]:
from datetime import date
current_date = date.today().strftime("%Y-%m-%d")


# Iterate over the anomaly flags and insert them into the Snowflake table
for index, flag in enumerate(anomaly_flags):
    meter_id = views[index]
    anomaly_flag_value = flag
    
    # Construct the SQL query
    query = f"INSERT INTO anomalies (meterId, anomaly_flag_value, date) VALUES ('{meter_id}', {anomaly_flag_value}, '{current_date}')"

    # Execute the SQL query
    cursor.execute(query)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()